<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_text_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [12]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo/cloned-repo
cloned-repo	     dataset.py		 EAST_text_detector.ipynb  model.py
dataset_original.py  EAST_dataset.ipynb  loss.py		   train.py
CPU times: user 118 ms, sys: 27.3 ms, total: 145 ms
Wall time: 7.25 s


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from dataset_original import custom_dataset

# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Annotations'


# icdar dataset
# image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
# label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

428


In [0]:
def extract_vertices(lines):
	'''extract vertices info from txt lines
	Input:
		lines   : list of string info
	Output:
		vertices: vertices of text regions <numpy.ndarray, (n,8)>
		labels  : 1->valid, 0->ignore, <numpy.ndarray, (n,)>
	'''
	labels = []
	vertices = []
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		label = 0 if '##::' in line else 1
		labels.append(label)
	return np.array(vertices), np.array(labels)

In [0]:
from dataset_original import *

In [17]:
img = Image.open(image_address+'/0.jpg')

with open(label_address+'/0.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[-27.95817481 259.01735746 209.1379291  118.39098975  88.1314165
 192.74665186 166.12367056 261.62996655]
--------------------
[154.5562676  465.84930483 401.137368   329.08821645  94.45545883
 201.18173565 171.04379279 279.90704907]
--------------------
[351.11417886 609.4106503  510.58394653 520.463317    95.68495589
 201.35733311 176.83923111 274.45818969]
--------------------
[1 1 1]


In [18]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(814.)

## Training

In [19]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(6123.)
CPU times: user 4.65 s, sys: 3.6 s, total: 8.25 s
Wall time: 4.2 s


In [0]:
from model import EAST

model = EAST()

In [21]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [22]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.94193238, angle loss is 0.83617729, iou loss is 1.76681161


tensor(11.0705, grad_fn=<AddBackward0>)

## Training Full Batch

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5//2], gamma=0.1)
criterion = Loss()

In [0]:
import os
import time

def train(train_img_path, train_gt_path, pths_path, batch_size, lr, num_workers, epoch_iter, interval):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = EAST()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[epoch_iter//2], gamma=0.1)
  
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      if (epoch + 1) % interval == 0:
        state_dict = model.module.state_dict() if data_parallel else model.state_dict()
        torch.save(state_dict, os.path.join(pths_path, 'model_epoch_{}.pth'.format(epoch+1)))

In [0]:
training = train(image_address, label_address, 'models', batch_size=4, lr=1e-3, num_workers=0, epoch_iter=1, interval=5)

428


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.92798018, angle loss is 0.80903655, iou loss is 2.18565321
Epoch is [1/1], mini-batch is [1/107], time consumption is 0.63570571, batch_loss is 11.20399857
classify loss is 0.85519922, angle loss is 0.60309690, iou loss is 1.61759293
Epoch is [1/1], mini-batch is [2/107], time consumption is 0.51502895, batch_loss is 8.50376129
classify loss is 0.90211928, angle loss is 0.86482328, iou loss is 1.91286838
Epoch is [1/1], mini-batch is [3/107], time consumption is 0.55839729, batch_loss is 11.46322060
classify loss is 0.89295387, angle loss is 0.77143377, iou loss is 1.83569467
Epoch is [1/1], mini-batch is [4/107], time consumption is 0.59625363, batch_loss is 10.44298649
classify loss is 0.90469599, angle loss is 0.78680545, iou loss is 1.74911380
Epoch is [1/1], mini-batch is [5/107], time consumption is 0.55348015, batch_loss is 10.52186394
classify loss is 0.90775156, angle loss is 0.88408160, iou loss is 1.75043941
Epoch is [1/1], mini-batch is [6/107], time cons